In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

In [ ]:
data=pd.read_csv('english_data.csv')['seq']

In [ ]:
text = open('choruses_lyrics.txt', 'rb').read().decode(encoding='utf-8')

In [ ]:
text[0:5000]

'They call me Mr. Pitiful\nBaby that\'s my name now\nThey call me Mr. Pitiful that\'s how I got my fame \n\nBut people just don\'t seem to understand \nHow someone can feel so blue\nThey call me Mr Pitiful cause I lost someone just like you.\n\nThe call me Mr. Pitiful\nThis everybody know now\nThey call me Mr. Pitiful Most every place I go. \n\nBut nobody seems to understand now, \nHow can a man sing such a sad song\nWhen he lost everything that he had.\n\nHow can I explain to you\nHow somebody con get so very blue\nHow can I tell you about my past \nIf all things won\'t end. \n\nMr Pitiful\nThat\'s my name now\nThey call me Mr. Pitiful that\'s how I got my fame \n\nBut nobody seems to understand \nWhat make a man can feel so very blue\nThey call me Mr. Pitiful cause I m in love with you\n\nCan I explain to you\nEverything is going wrong\nI\'ve lost everything I had \nI have to sing this sad song \nTo get back to her \nAnd I m gonna sing this song to you \nAnd I want you \nAnd I want y

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

95 unique characters


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BATCH_SIZE = 100

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(100, 100), dtype=tf.int64, name=None), TensorSpec(shape=(100, 100), dtype=tf.int64, name=None))>

In [ ]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 768

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(100, 100, 95) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (100, None, 256)          24320     
                                                                 
 gru_9 (GRU)                 (100, None, 768)          2363904   
                                                                 
 dense_9 (Dense)             (100, None, 95)           73055     
                                                                 
Total params: 2461279 (9.39 MB)
Trainable params: 2461279 (9.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([80, 48,  4, 19, 78, 23, 89, 21,  8,  5, 74, 82, 87, 25, 23, 82, 81,
        5, 94, 45, 94,  8,  6, 46, 10, 46, 11, 64, 64, 45, 25, 25, 74, 86,
       48, 26, 25, 54, 59, 17, 78, 27, 64, 63, 68, 89, 55, 73, 42, 93, 60,
       64, 68, 87, 87, 87, 18, 63, 41,  9,  2, 35, 27, 19, 83, 64, 62, 36,
       49, 81, 72, 42, 85,  3,  3, 76, 27, 64, 32, 82, 40, 63, 10, 92,  0,
       26, 57, 90,  0, 67, 34, 40, 84, 21, 72, 57, 26, 13, 82, 79])

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)


In [ ]:
model.compile(optimizer='nadam', loss=loss, metrics=['accuracy'])

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 50

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
803/803 [==============================] - 81s 66ms/step - loss: 1.8668 - accuracy: 0.4598
Epoch 2/20
803/803 [==============================] - 51s 61ms/step - loss: 1.4098 - accuracy: 0.5721
Epoch 3/20
803/803 [==============================] - 51s 61ms/step - loss: 1.3255 - accuracy: 0.5959
Epoch 4/20
803/803 [==============================] - 51s 61ms/step - loss: 1.2779 - accuracy: 0.6102
Epoch 5/20
803/803 [==============================] - 51s 61ms/step - loss: 1.2432 - accuracy: 0.6211
Epoch 6/20
803/803 [==============================] - 51s 61ms/step - loss: 1.2162 - accuracy: 0.6300
Epoch 7/20
803/803 [==============================] - 51s 62ms/step - loss: 1.1943 - accuracy: 0.6372
Epoch 8/20
803/803 [==============================] - 51s 62ms/step - loss: 1.1768 - accuracy: 0.6431
Epoch 9/20
803/803 [==============================] - 51s 61ms/step - loss: 1.1622 - accuracy: 0.6479
Epoch 10/20
803/803 [==============================] - 52s 61ms/step - loss: 1.150

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (1, None, 256)            24320     
                                                                 
 gru_10 (GRU)                (1, None, 768)            2363904   
                                                                 
 dense_10 (Dense)            (1, None, 95)             73055     
                                                                 
Total params: 2461279 (9.39 MB)
Trainable params: 2461279 (9.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.save('./model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def generate_text(model, start_string,t):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    temperature = t

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Call me",t=0.4))

Call me and I'll be there

Can't you see the witch with my heart and so well

Well, I can go on and on and on
You got me singing the same old song
I want to leave you baby I can't let go
I can't let go
I can't let go
I can't let go
I can't let go
I can't get no satisfaction
It's not what you see
It's too bad to the love of my life

I wanna be your and harder and a dog
It's a time where I have nothing left to do things
For your love
I'll show you the way it is

I want to be with you
I love you to me to you
You gotta know what I'm thinking of you
And the what will be an answer
The way you hang me one more time
To get it on that star
Your not too long

I thought that I was wrong
I believe in a world of courtes
I'm gonna stand alone and the only one who had a double show
I want to see you sad
If I had to get out of my mind

But that was just a bad car
And the war is out there
She said she was born to be a barefoot ballad you better change your mind
When you were my own
And the world is so 

In [ ]:
print(generate_text(model, start_string=u"So", t=0.2))

Sophis and the moonlight
And the storm has gone and space
When I come through the night
And I will leave you baby I ask myself
And I want to love you

I know you're not alone

I want to live fast, love hard, die young and bought a bride
But I was standing next to me
I'm still in love with a sound of mine

I want to see your face in the morning
I want to see your face in every kind of light
In the morning she was always on my mind
The love that I should have been all along

I want to see your face in every kind of light
The way you have to go to my head
When I get home to you 

I know I can't live without you baby
I'm gonna stand the party that I need to say
If you want to be my no. 1
Let me know your future plans.

I want to see your face in the club and scream and start to shine
I got a short of this is the only one who can love ya like that
Oh my heart is to be saved by the same old song
I have no fear of love
I have no fear in my heart and some people like the way you talk
And I wan

In [ ]:
print(generate_text(model, start_string=u"Why", t=0.8))

Why (what I say)
It's your own in bed
You said, "Funt, now

Too furth girl now the sweetness of the night
I was always running out of time
Sharing out of time

So quickly to live without me
And it won't be lonely 

Oh Lordy what can I say to love (just a little bit) 

The same old sound, dark like craise the Lang I Crew does the joy, we the love that you have to
So lonely nights, come and baby mine.
You know I said "Hey, and don't you mind.
Come on say hello
Soon I'll never be

I know I wouldn't believe to-dance all night, darling

The witch is open winds
For something
Every earth
Can we all all be a sad at the New Orroram!
We gonna happen, and the heavens dear, what have we found the critical cookin'
All over the party 
We'll party chairs started rollin' rollin' rollin straight to your heart 
Did without him
'Cause if you lives its dig beautiful towards the river
Check it out mind and complains
When rap what I do

This is the one that be free
 Can't go on living to the left or leaning

In [ ]:
print(generate_text(model, start_string=u"Let me", t=0.7))

Let me the love I know
That I'd be the one who can love ya

Hanging off, start to learn to pass away
Just because there's no one at all

The way there was another pillow
Stormy weather
To have to spend together
Tell me what makes you think about your love

I'm gonna get you some more
Well it was the price of real

Cause I'm in love with you

You're so easy
To reach out and the love we should be
The real me up in love with me
Away from me to my lover
Can't you feel good to me

Something more than you keep it louder, love is the one to not seem to forgive me
I'd never search your soul
It's gonna be lonely, darling
I saw the green and blue
I have to give you all the time
Well, how we there was a sign of a shoulder
Music on the long way down the road.
Give me your face as despair
I don't want to see you playin' all around I saw your face
But I don't wanna see that much touch out of you
Sometimes love of my inner to tap the monkey cracked eye
She'll always find the power and called you on t